<목차>
---
1. Gradient Descent
2. Training, Validation, Test Data & Overfitting
3. Softmax Regression
    * 소프트맥스 회귀
    * 크로스 엔트로피 손실 함수
    * MNIST 데이터셋
    * One-hot Encoding
4. Softmax Regression을 이용한 MNIST 숫자 분류기 구현

# 1. Gradient Descent (경사하강법)
---
손실 함수를 최소화하는 대표적인 알고리즘  
- Batch Gradient Descent  
    - **전체** traing set을 하나의 batch로 만듬  
    - 단점: 시간이 오래 걸림
- Stochastic Gradient Descent   
    - 한 스텝 진행할 때 **1개**의 training set만 사용  
    - 단점: 파라미터를 한 번 업데이트할 때, 전체 set의 특성을 고려하지 않음  
- Mini-Batch Gradient Descent
    - 위 단점들을 해결  
    ex) 전체 set이 1000개라면 100개씩 mini-batch를 만들어 파라미터를 업데이트




# 2. Training, Validation, Test Data & Overfitting
---
# 3. Softmax Regression
---
- Softmax Regression  
  - 레이블을 분류하기 위한 목적  
  - Softmax 함수는 정규화함수로써 출력값들의 합을 1로 만든다.  
  - 모델의 출력값이 label에 대한 확률을 나타낸다.  

- Cross-Entropy 손실 함수
  - 분류 문제에 많이 사용됨

- MNIST 데이터셋
  - 60,000장의 training set, 10,000장의 test set  
  - 0~9사이의 28*28 크기의 필기체 이미지로 구성

- One-hot Encoding
  - 범주형 값을 이진화된 값으로 바꿔서 표현

# 4. Softmax Regression을 이용한 MNIST 숫자 분류기 구현 
---
데이터 다운로드

In [ ]:
#텐서플로 버전2
import tensorflow as tf

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# 이미지들을 float32 데이터 타입으로 변경
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')

# 28*28 형태의 이미지를 784차원으로 flattening 
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])

# [0, 255] 사이의 값을 [0, 1]사이의 값으로 Normalize
x_train, x_test = x_train / 255., x_test / 255.

# 레이블 데이터에 one-hot encoding을 적용
y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)

# train data
# tf.data API를 이용해서 데이터를 섞고 batch 형태로 가져옵니다.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(60000).batch(100)
train_data_iter = iter(train_data)

모델 정의

In [ ]:
class SoftmaxRegression(tf.keras.Model):
  def __init__(self):
    super(SoftmaxRegression, self).__init__()
    self.softmax_layer = tf.keras.layers.Dense(10,
                                               activation=None,
                                               kernel_initializer='zeros',
                                               bias_initializer='zeros')

  def call(self, x):
    logits = self.softmax_layer(x)

    return tf.nn.softmax(logits)

손실 함수 정의

In [ ]:
def cross_entropy_loss(y_pred, y):
  return tf.reduce_mean(-tf.reduce_sum(y * tf.math.log(y_pred), axis=[1]))
   #return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logtis, labels=y)) 

다른 방법: TensorFlow API를 이용한 구현  
**tf.nn.softmax_cross_entropy_with_logits_v2**(labels = None, logits = None, name = None)  
- labels: 정답 레이블  
- logits: softmax 함수를 적용해서 정규화하기 전 모델의 출력값 logits


최적화 함수 정의

In [ ]:
optimizer = tf.optimizers.SGD(0.5)

def train_step(model, x, y):
  with tf.GradientTape() as tape:
    y_pred = model(x)
    loss = cross_entropy_loss(y_pred, y)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

모델 성능 출력 함수

In [ ]:
def compute_accuracy(y_pred, y):
  correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  return accuracy

실행

In [ ]:
# SoftmaxRegression 모델을 선언
SoftmaxRegression_model = SoftmaxRegression()

# 1000번 반복을 수행하면서 최적화를 수행
for i in range(1000):
  batch_xs, batch_ys = next(train_data_iter)
  train_step(SoftmaxRegression_model, batch_xs, batch_ys)

# 학습이 끝나면 학습된 모델의 정확도를 출력
print("정확도(Accuracy): %f" % compute_accuracy(SoftmaxRegression_model(x_test), y_test))

정확도(Accuracy): 0.920500
